In [2]:
import torch
from torch import nn
from torch.nn import functional as F
net= nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
X=torch.rand(size=(2,20))
print(X)

tensor([[0.1288, 0.2091, 0.9733, 0.2143, 0.7007, 0.6389, 0.7280, 0.5466, 0.2294,
         0.6255, 0.3928, 0.9166, 0.6962, 0.1752, 0.1373, 0.2920, 0.5996, 0.1323,
         0.7602, 0.6577],
        [0.0407, 0.0649, 0.9489, 0.7724, 0.9767, 0.2141, 0.0494, 0.5860, 0.2566,
         0.4875, 0.5900, 0.8850, 0.0259, 0.3350, 0.5107, 0.2136, 0.2897, 0.9464,
         0.5961, 0.1148]])


In [3]:
net(X)

tensor([[ 0.0660,  0.1666,  0.0404,  0.0347, -0.0414, -0.1084, -0.3117, -0.0910,
         -0.0873,  0.0236],
        [ 0.0655,  0.2767,  0.1000, -0.0145, -0.0178, -0.1788, -0.3581, -0.0654,
          0.1606, -0.0033]], grad_fn=<AddmmBackward0>)

In [10]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden=nn.Linear(20,256)
        self.out=nn.Linear(256,10)

    def forward(self,X):
        return self.out(F.relu(self.hidden(X)))

In [11]:
net=MLP()
net(X)

tensor([[-0.1748, -0.1956,  0.1434,  0.3129,  0.0700, -0.0669, -0.1376,  0.1553,
         -0.0726,  0.1579],
        [-0.1085, -0.2456,  0.2598,  0.4173,  0.1360, -0.1070, -0.0219,  0.1750,
         -0.0247,  0.0827]], grad_fn=<AddmmBackward0>)

In [16]:
class MySequential(nn.Module):
    def __init__(self,*args):
        super().__init__()
        for idx,module in enumerate(args):
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)]=module
    def forward(self,X):
        for block in self._modules.values():
            X=block(X)
        return X

In [17]:
net = MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

tensor([[ 2.2698e-02,  2.7680e-02, -2.5772e-01, -3.9994e-03,  1.1858e-01,
         -2.4898e-01, -2.6041e-02,  1.6477e-02, -1.7683e-02, -2.7140e-01],
        [ 6.5375e-03,  1.8121e-03, -3.1809e-01,  4.0880e-02,  1.1717e-01,
         -3.5158e-01, -2.1721e-01,  3.4784e-02, -1.0558e-04, -6.2832e-02]],
       grad_fn=<AddmmBackward0>)

In [18]:
nn.Sequential??

In [21]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight=torch.rand((20,20),requires_grad=False)
        self.linear=nn.Linear(20,20)
    def forward(self,X):
        X=self.linear(X)
        X=F.relu(torch.mm(X,self.rand_weight)+1)
        X=self.linear(X)
        while X.abs().sum()>1:
            X/=2
        return X.sum()

In [23]:
net=FixedHiddenMLP()
net(X)

tensor(-0.0252, grad_fn=<SumBackward0>)

In [27]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net=MySequential(nn.Linear(20,64),nn.ReLU(),nn.Linear(64,32)
                               ,nn.ReLU())
        self.linear=nn.Linear(32,16)
    def forward(self,X):
        return self.linear(self.net(X))
chimera=MySequential(NestMLP(),nn.Linear(16,20),FixedHiddenMLP())
chimera(X)

tensor(0.3446, grad_fn=<SumBackward0>)